In [20]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [21]:
DAYS_PREDICT = 15

In [22]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_btc = yf.download("BTC-USD", data_init, date_today)

df_btc.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [23]:
df_btc.reset_index(inplace=True)
df_btc.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2917,2022-09-12,21770.148438,22439.181641,21603.896484,22370.449219,22370.449219,50212088965
2918,2022-09-13,22371.480469,22673.820312,20062.669922,20296.707031,20296.707031,51091116622
2919,2022-09-14,20184.554688,20467.201172,19793.396484,20241.089844,20241.089844,37872380889
2920,2022-09-15,20242.289062,20318.166016,19636.734375,19701.210938,19701.210938,36389011503
2921,2022-09-16,19704.005859,19870.628906,19400.076172,19772.583984,19772.583984,30123362273


In [24]:
df = df_btc[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [25]:
df.tail()

,ds,y
2917,2022-09-12,22370.449219
2918,2022-09-13,20296.707031
2919,2022-09-14,20241.089844
2920,2022-09-15,19701.210938
2921,2022-09-16,19772.583984


In [26]:
df.isnull().sum().sum()
df['ds'].isnull().sum()
df['y'].isnull().sum()

0

In [27]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [28]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:33:03 - cmdstanpy - INFO - Chain [1] start processing
00:33:05 - cmdstanpy - INFO - Chain [1] done processing


In [29]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
2932,2022-09-27
2933,2022-09-28
2934,2022-09-29
2935,2022-09-30
2936,2022-10-01


In [30]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2014-09-17,174.417686,-3170.964393,3686.861895,174.417686,174.417686,-0.059707,-0.059707,-0.059707,0.002044,0.002044,0.002044,-0.061751,-0.061751,-0.061751,0.0,0.0,0.0,164.003718
1,2014-09-18,174.882035,-3376.860976,3420.061077,174.882035,174.882035,-0.060297,-0.060297,-0.060297,-0.002394,-0.002394,-0.002394,-0.057903,-0.057903,-0.057903,0.0,0.0,0.0,164.337116
2,2014-09-19,175.346384,-3180.926803,3383.604293,175.346384,175.346384,-0.054923,-0.054923,-0.054923,-0.001527,-0.001527,-0.001527,-0.053396,-0.053396,-0.053396,0.0,0.0,0.0,165.715880
3,2014-09-20,175.810733,-3258.168700,3390.652228,175.810733,175.810733,-0.047307,-0.047307,-0.047307,0.000888,0.000888,0.000888,-0.048195,-0.048195,-0.048195,0.0,0.0,0.0,167.493573
4,2014-09-21,176.275082,-2949.667211,3416.419621,176.275082,176.275082,-0.041358,-0.041358,-0.041358,0.000913,0.000913,0.000913,-0.042271,-0.042271,-0.042271,0.0,0.0,0.0,168.984622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,2022-09-27,30849.943298,27960.755619,34490.297863,30849.943298,30849.943298,0.008747,0.008747,0.008747,-0.000963,-0.000963,-0.000963,0.009710,0.009710,0.009710,0.0,0.0,0.0,31119.790822
2933,2022-09-28,30812.870767,28201.537947,34781.151811,30812.870767,30812.870767,0.023256,0.023256,0.023256,0.002044,0.002044,0.002044,0.021212,0.021212,0.021212,0.0,0.0,0.0,31529.454850
2934,2022-09-29,30775.798235,28245.544540,35454.229554,30775.798235,30775.798235,0.031119,0.031119,0.031119,-0.002394,-0.002394,-0.002394,0.033513,0.033513,0.033513,0.0,0.0,0.0,31733.507667
2935,2022-09-30,30738.725704,28926.368454,35291.514770,30738.725704,30738.725704,0.045063,0.045063,0.045063,-0.001527,-0.001527,-0.001527,0.046590,0.046590,0.046590,0.0,0.0,0.0,32123.898147


In [31]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
2932,2022-09-27,27960.755619,34490.297863,31119.790822
2933,2022-09-28,28201.537947,34781.151811,31529.454850
2934,2022-09-29,28245.544540,35454.229554,31733.507667
2935,2022-09-30,28926.368454,35291.514770,32123.898147
2936,2022-10-01,29055.673992,35832.296307,32583.625692


In [32]:
# grafico 
#plot.plot(model, predict)

In [33]:
# Salvar modelo
with open('model_btc_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)